# Comienza el análisis ETL USER ITEMS

Primero, importo las librerias necesarias para hacer el respectivo análisis.

In [1]:
import pandas as pd
import numpy as np
import ast
import gzip

Se construye la funcion 'descomprimir_json(archivo)', toma un archivo comprimido en formato JSON utilizando gzip, lo descomprime y devuelve una lista de diccionarios, donde cada diccionario representa un objeto JSON dentro del archivo comprimido. Utiliza la biblioteca gzip para descomprimir el archivo y ast.literal_eval para convertir cada línea del archivo descomprimido en un diccionario de Python.

In [2]:
def descomprimir_json(Archivo):
    with gzip.open(Archivo, 'rt', encoding='utf-8') as Archivo:
        return [ast.literal_eval(line.strip()) for line in Archivo]

Cargamos datos desde un archivo JSON comprimido, los convierte en un DataFrame de pandas y luego muestra las primeras tres filas de ese DataFrame.

In [3]:
df_items_json = descomprimir_json('users_items.json.gz')
df_items = pd.DataFrame(df_items_json)
df_items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


toma una columna del DataFrame df_items que contiene listas, las extrae y crea un nuevo DataFrame df_items_ donde cada elemento de las listas originales se convierte en una fila.

In [4]:
f=[]
for i in df_items.values:
    f.append(i[4])

df_items_ = pd.DataFrame([items for sublist in f for items in sublist])

Crea un nuevo DataFrame donde cada fila contiene un solo elemento de la lista original en 'items', y se replica el 'user_id' correspondiente para cada elemento.

In [8]:
f = []

for i in df_items['items'].values:
    f.extend(i)

df_items_user = pd.DataFrame({'items': f})
df_items_user['user_id'] = df_items['user_id'].repeat(df_items['items'].str.len()).reset_index(drop=True)


In [9]:
df_items_user

,items,user_id
0,"{'item_id': '10', 'item_name': 'Counter-Strike...",76561197970982479
1,"{'item_id': '20', 'item_name': 'Team Fortress ...",76561197970982479
2,"{'item_id': '30', 'item_name': 'Day of Defeat'...",76561197970982479
3,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",76561197970982479
4,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",76561197970982479
...,...,...
5153204,"{'item_id': '346330', 'item_name': 'BrainBread...",76561198329548331
5153205,"{'item_id': '373330', 'item_name': 'All Is Dus...",76561198329548331
5153206,"{'item_id': '388490', 'item_name': 'One Way To...",76561198329548331
5153207,"{'item_id': '521570', 'item_name': 'You Have 1...",76561198329548331


Convertimos los datos JSON anidados en la columna 'items' del DataFrame df_items_user en un DataFrame plano df_items, donde cada elemento JSON se convierte en una fila y cada clave en el JSON se convierte en una columna.

In [10]:
df_items = pd.json_normalize(df_items_user['items'])

Asignamos la columna 'user_id' del DataFrame df_items_user a una nueva columna llamada 'user_id' en el DataFrame df_items. La asignación se realiza de tal manera que cada valor en la columna 'user_id' del DataFrame df_items_user se asigna a todas las filas correspondientes en el DataFrame df_items.

In [11]:
df_items['user_id'] = df_items_user['user_id']

In [12]:
df_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479
...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331
5153205,373330,All Is Dust,0,0,76561198329548331
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331


Guardamos en un archivo csv, el dataframe 'df_items'

In [15]:
df_items.to_csv('user_items_2.csv', index=False)
df_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479
...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331
5153205,373330,All Is Dust,0,0,76561198329548331
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331
